In [1]:
from openai import OpenAI
import numpy as np

In [6]:
client = OpenAI(api_key="sk-svcacct-oD2ywALJLsVNQwU3GZ-Lvl-6d53TV1hwjr7DKEfvT07SSYDzFHvmTo3Fx9gp-5KT3BlbkFJsc-MOn7QvehX14nXoONvR0tKAQ8_XBfDFywvS3CVLyL4QaCm9Pklco6v0i3hhAA")

In [7]:
with open("messi.txt", "r") as f:
    text = f.read()
    print(text)

Lionel Andrés Messi[note 1] (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ⓘ; born 24 June 1987), also known as Leo Messi, is an Argentine professional footballer who plays as a forward for and captains both Major League Soccer club Inter Miami and the Argentina national team. Widely regarded as one of the greatest players of all time, Messi set numerous records for individual accolades won throughout his professional footballing career such as eight Ballon d'Or awards and eight times being named the world's best player by FIFA.[note 2] He is the most decorated player in the history of professional football having won 45 team trophies,[note 3] including twelve league titles, four UEFA Champions Leagues, two Copa Américas, and one FIFA World Cup. Messi holds the records for most European Golden Shoes (6), most goals for a single club (672, with Barcelona), most goals (474), hat-tricks (36) and assists (192) in La Liga, most matches played (39), assists (18) and goal contributions (34) 

In [8]:
CHUNNK_SIZE = 50
chunks = []
current_place = 0
while current_place < len(text):
    current_chunk = text[current_place : current_place + CHUNNK_SIZE]
    chunks.append(current_chunk)
    current_place += CHUNNK_SIZE
print(chunks)

['Lionel Andrés Messi[note 1] (Spanish pronunciation', ': [ljoˈnel anˈdɾes ˈmesi] ⓘ; born 24 June 1987), a', 'lso known as Leo Messi, is an Argentine profession', 'al footballer who plays as a forward for and capta', 'ins both Major League Soccer club Inter Miami and ', 'the Argentina national team. Widely regarded as on', 'e of the greatest players of all time, Messi set n', 'umerous records for individual accolades won throu', 'ghout his professional footballing career such as ', "eight Ballon d'Or awards and eight times being nam", "ed the world's best player by FIFA.[note 2] He is ", 'the most decorated player in the history of profes', 'sional football having won 45 team trophies,[note ', '3] including twelve league titles, four UEFA Champ', 'ions Leagues, two Copa Américas, and one FIFA Worl', 'd Cup. Messi holds the records for most European G', 'olden Shoes (6), most goals for a single club (672', ', with Barcelona), most goals (474), hat-tricks (3', '6) and assists (192) in La

In [9]:
embedded_chunks = []
for chunk in chunks:
    response = client.embeddings.create(
        input=chunk,
        model="text-embedding-3-small"
    )
    embedded_chunks.append( (chunk, response.data[0].embedding) )
print(embedded_chunks)

[('Lionel Andrés Messi[note 1] (Spanish pronunciation', [0.0384596511721611, -0.06454427540302277, -0.04185671731829643, -0.002527579665184021, -0.03882362321019173, 0.018795082345604897, 0.012071720324456692, 0.010848372243344784, -0.0354669988155365, -0.01616640016436577, -0.07137884944677353, 0.009271162562072277, -0.01016592513769865, -0.032150812447071075, 0.02778315544128418, 0.05309939384460449, -0.05896337702870369, -0.012920986860990524, 0.00016381875320803374, -0.04962144419550896, 0.030108528211712837, -0.012364919297397137, 0.016459599137306213, 0.01855243369936943, 0.02539712004363537, 0.0034501461777836084, -0.05111777037382126, 0.011818962171673775, -0.0009175114100798965, 0.01567099429666996, 0.07651489228010178, -0.024487191811203957, 0.010373187251389027, 0.00017298123566433787, -0.012860325165092945, -0.017248203977942467, -0.029744558036327362, -0.019523024559020996, -0.004845370072871447, -0.04460672661662102, -0.018350228667259216, -0.037893474102020264, 0.0188759

In [10]:
def cosine_simularity(A, B):
    return np.dot(A,B) / ( np.linalg.norm(A) * np.linalg.norm(B) )

In [11]:
question = "Who is the best footballer in the world?"

In [12]:
question_embedding = client.embeddings.create(
        input=question,
        model="text-embedding-3-small"
    ).data[0].embedding
print(question_embedding)

[0.030270060524344444, -0.012901217676699162, 0.005253246985375881, 0.05039330944418907, -0.002194768749177456, -0.01476588286459446, 0.011027085594832897, 0.04111730679869652, -0.020766889676451683, -0.017700130119919777, -0.011254253797233105, 0.0077094947919249535, -0.045849960297346115, 0.008679688908159733, 0.019877150654792786, -0.026995062828063965, -0.05228637158870697, 0.006975933443754911, 0.0339236706495285, -0.014614438638091087, 0.039905745536088943, -0.00919081550091505, 0.01540952455252409, 0.008925787173211575, 0.06970254331827164, -0.00449838861823082, -0.03434014320373535, -0.013497531414031982, -0.04895458370447159, 0.06167595833539963, 0.042291004210710526, -0.010506493970751762, 0.019157787784934044, -0.017586546018719673, -0.029077431187033653, -0.03861846402287483, -0.011869498528540134, -0.011301579885184765, -0.003928103484213352, 0.010752592235803604, 0.0396028570830822, 0.02644607611000538, -0.04952250048518181, 0.005982075817883015, 0.027752287685871124, -0.

In [13]:
best_cosine_similarity = 0
index = 0
for i in range(len(embedded_chunks)):
    similarity = cosine_simularity(question_embedding, embedded_chunks[i][1])
    if similarity > best_cosine_similarity:
        best_cosine_similarity = similarity
        index = i

In [14]:
print(f"Relevant information: {embedded_chunks[index][0]}")

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user", "content": f"User question: {question}. Potentially useful information: {embedded_chunks[index][0]}"
        }
    ]
)

Relevant information: ed the world's best player by FIFA.[note 2] He is 


In [15]:
print(completion.choices[0].message.content)

Determining the "best" footballer in the world is subjective and can vary depending on personal opinions, team allegiances, and the period in question. However, historically, players like Lionel Messi and Cristiano Ronaldo have frequently been considered among the best in the world and have won numerous Ballon d'Or awards, which annually recognize the best male player in the world.

Currently, as of 2023, Lionel Messi, after leading Argentina to a World Cup victory in 2022 and achieving a successful career filled with individual and team accolades, is often cited as the best footballer. However, other players such as Kylian Mbappé, Erling Haaland, and others have been emerging as top contenders, given their impressive performances and impact on the game.

FIFA's The Best award and France Football's Ballon d'Or are two prestigious awards that recognize outstanding footballers annually, which consistently highlight the top performers in world football.
